# Library

In [1]:
import os, sys, glob, time, json, copy, random, datetime
import scipy.sparse as sp
import argparse
import warnings
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io as scio
from pprint import pformat
import openpyxl
from collections import defaultdict, Counter, namedtuple

In [2]:
import pytorch_lightning as pl
from pytorch_lightning.trainer import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.utilities import move_data_to_device

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor
from torch.nn import Parameter as Param
from torch.utils.data import DataLoader
from torch.utils.data.sampler import BatchSampler, RandomSampler, SequentialSampler

In [4]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc, accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import KFold

In [5]:
from torch_geometric.data import Data

In [6]:
from src import metric_fn
from src.utils import init_logger, logger
from src.dataloader import CVDataset, DRDataset
from src.DRHGCN.model import DRHGCN

In [7]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
sns.set(style='darkgrid')
warnings.simplefilter('ignore')

In [8]:
print(torch.__version__)

1.12.1+cu102


In [9]:
print(pl.__version__)

1.7.7


## MyCodes

In [10]:
def get_parent_path(pth, depth=1):
    for _ in range(depth):
        pth = os.path.dirname(pth)
    return pth

In [11]:
def glob_files(pth):
    for f in glob.glob(os.path.join(pth, '*')):
        print(f)

In [12]:
def get_pair(arr):
    return set([(l[0], l[1]) for l in arr.T.tolist()])

In [13]:
def union_set(s1,s2):
    return s1.union(s2)

In [14]:
def measure_time(txt, time_begin, time_fin):
    time = time_fin - time_begin
    print("{}: {:0.3f}".format(txt, time))

In [15]:
def count_unique_ids(var):
    if isinstance(var, pd.DataFrame):
        ret = len(var['MyID'].unique())
        return ret
    else:
        try:
            ret = len(set([x['id'] for x in var.values()]))
            return ret
        except:
            raise Exception("Dtype error: Node file")

In [16]:
def load_nodes(path, keys):
    # load node dataframes
    nodes = dict()
    for key in keys:
        filepath = os.path.join(path, "Dataset", "Node", "{}_wID.csv".format(key))
        if os.path.exists(filepath):
            nodes[key] = pd.read_csv(filepath)
        else:
            print("{} node file does not exist".format(key))
    return nodes

In [17]:
def load_edges(path, unweighted_keys, weighted_keys):
    edge_indexes = dict()
    edge_weights = dict()
    
    # load network data (unweighted)
    for key in unweighted_keys:
        filepath = os.path.join(path, "Dataset", 'Edge', "{}.npy".format(key))
        if os.path.exists(filepath):
            edge = sp.coo_matrix(np.load(filepath))
            edge = Tensor(np.array([edge.row, edge.col])).long()
            edge_indexes[key] = edge
        else:
            print("{} network file does not exist".format(key))
    
    # load network data (weighted)
    for key in weighted_keys:
        filepath = os.path.join(path, "Dataset", 'Edge', "{}.npy".format(key))
        if os.path.exists(filepath):
            edge = np.load(filepath)
            row, col = edge.nonzero()
            edge_indexes[key] = Tensor([row, col]).long()
            edge_weights[key] = Tensor(np.array([edge[i][j] for i,j in zip(row, col)]))
        else:
            print("{} network file does not exist".format(key))
    
    return edge_indexes, edge_weights

In [18]:
def describe_dataset(data, nodes, edges, edge_symbols, conf):
    
    task_type = conf['task_type']
    
    line = "#"*40
    print(line)
    
    # Dataset
    print('Node >>')
    for key in nodes:
        print('\t{}: {}'.format(key, data['n_{}'.format(key)]))
        
    print("Edge >>")
    for key in edges:
        src, tar, *diff = key.split('_')
        if src==tar:
            print('\t{}: {}'.format(key, data["{}_edge_index".format(edge_symbols[key])].shape[1]))
        else:
            print('\t{}: {}'.format(key, data["{}_edge_index".format(edge_symbols[key])].shape[1]//2))
    
    # Link prediction
    print(line)
    print("Task type: {}".format(task_type))
    print("Internal drug nodes: {}".format(len(data.int_drug_index)))
    print("External drug nodes: {}".format(len(data.ext_drug_index) if data.ext_drug_index is not None else 0))
    
    print(line)
    for i in range(conf['cv']):
        print("Train (cv-{}): {} >> {}".format(i, "INT", data.train_edge_index[i].shape[1]))
        print("Valid (cv-{}): {} >> {}\n".format(i, "INT" if task_type=='transductive' else "EXT", data.valid_edge_index[i].shape[1]))
    print("Test:  {} >> {}".format("INT" if task_type=='transductive' else "EXT", data.test_edge_index.shape[1]))
    print(line)

In [19]:
# Data partitioning randomly
def split_train_test_edges(index, train_ratio=0.8, cv=1):
    n_edge = index.shape[1]
    ret_train, ret_valid = [], []
    
    if cv==1:
        rd = np.random.binomial(1, train_ratio, n_edge)
        train_mask = rd.nonzero()[0]
        valid_mask = (1-rd).nonzero()[0]
        
        train_index = index[:, train_mask]
        valid_index = index[:, valid_mask]
        
        ret_train.append(train_index.long())
        ret_valid.append(valid_index.long())
    
    else:
        rd = np.random.permutation(np.arange(n_edge)%cv)
        for i in range(cv):
            train_index = index[:, rd!=i]
            valid_index = index[:, rd==i]
            ret_train.append(train_index.long())
            ret_valid.append(valid_index.long())
    
    return ret_train, ret_valid

In [20]:
# Data partitioning considering drug node order and ICD-11 classification 
def SepCategory_train_test_split(dz_edge_index, diz_df):
    
    train_edge_index = []
    test_edge_index = []
    
    # drug - disease pair
    dz_pairs = defaultdict(list)
    for src, tar in dz_edge_index.T:
        src = src.item()
        tar = tar.item()
        dz_pairs[src].append(tar)
    
    # Drugs: Degree >= 2
    drugs = dz_edge_index[0].tolist()
    drug_src_counts = Counter(drugs)
    test_drug_id = [k for k,v in drug_src_counts.items() if v>=2]
    
    # Disease: No overlapping ICD-11 classifications
    for did in test_drug_id:
        tars = dz_pairs[did]
        tmp = diz_df[diz_df['MyID'].isin(tars)]
        diz_cats = tmp['ICD-11_Category'].values

        if len(set(diz_cats)) <= 1:
            continue
        else:
            picked_cat = np.random.choice(diz_cats)
            picked_diz = tmp[tmp['ICD-11_Category']==picked_cat]
            for zid in picked_diz.MyID:
                test_edge_index.append([did, zid])
            
    # split train-test
    test_set = set([(x,y) for x,y in test_edge_index])
    for x,y in dz_edge_index.T:
        x = x.item()
        y = y.item()
        if (x,y) not in test_set:
            train_edge_index.append([x,y])
        else:
            pass
    
    train_edge_index = Tensor(np.array(train_edge_index)).T.long()
    test_edge_index = Tensor(np.array(test_edge_index)).T.long()
    
    return train_edge_index, test_edge_index

In [21]:
# Data partitioning for inductive link prediction task
def split_external_drug_edges(data, edge_symbols, train_ratio):
    
    # drug node split
    rd = np.random.binomial(1, train_ratio, data.n_drug)
    internal_index = np.array([i for i,x in enumerate(rd) if x==1])
    external_index = np.array([i for i,x in enumerate(rd) if x==0])
    
    internal_set = set(internal_index)
    external_set = set(external_index)
    
    # train-test-split for drug-disease network
    train_edge_index, test_edge_index = [], []
    for pair in data.d_z_edge_index.T.numpy():
        if pair[0] in internal_set:
            train_edge_index.append(pair)
        elif pair[0] in external_set:
            test_edge_index.append(pair)
        else:
            raise Exception("Data split error")
    train_edge_index = np.array(train_edge_index)
    test_edge_index = np.array(test_edge_index)
    
    # remove edges including external drugs
    ret = {}
    for key, val in edge_symbols.items():
        types = key.split('_')
        if val == 'd_z':
            ret[val] = train_edge_index
        elif types[0]=='drug':
            ret[val] = remove_edges(external_index, data['{}_edge_index'.format(val)], is_tar_drug=(types[1]=='drug'))
            
    # update edge index
    for key, val in edge_symbols.items():
        types = key.split('_')
        if key == 'drug_disease':
            data['{}_edge_index'.format(val)] = Tensor(train_edge_index.T).long()
        elif types[0] == 'drug':
            data['{}_edge_index'.format(val)] = Tensor(ret[val].T).long()
        else:
            pass
        
    train_edge_index = Tensor(train_edge_index.T).long()
    test_edge_index = Tensor(test_edge_index.T).long()
    return data, train_edge_index, test_edge_index, internal_index, external_index

In [22]:
def split_link_prediction_datas(data, nodes, edge_symbols, pred_type, cv=1):
    if pred_type == 'transductive':
        print("Transductive link prediction")
        # edge-wise split data
        train_edge_index, test_edge_index = SepCategory_train_test_split(data.d_z_edge_index, nodes['disease'])
        train_edge_index, valid_edge_index = split_train_test_edges(train_edge_index, train_ratio=0.8, cv=cv)
        
        data.int_drug_index = np.array([i for i in range(data.n_drug)])
        data.ext_drug_index = None
        data.train_edge_index = train_edge_index
        data.valid_edge_index = valid_edge_index
        data.test_edge_index = test_edge_index
        
    elif pred_type == 'inductive':
        print("Inductive link prediction")
        # node-wise split data (internal or external)
        data, int_edge_index, ext_edge_index, int_drug_index, ext_drug_index = split_external_drug_edges(data, edge_symbols, train_ratio=0.9)
        
        # edge-wise split data
        valid_ext_edge_index, test_ext_edge_index = split_train_test_edges(ext_edge_index, train_ratio=0.5)
        
        data.int_drug_index = int_drug_index
        data.ext_drug_index = ext_drug_index
        data.train_edge_index = int_edge_index
        data.valid_edge_index = valid_ext_edge_index
        data.test_edge_index = test_ext_edge_index
        
    else:
        raise Exception('Invalid data split type')
        
    return data

In [23]:
# Negative sampling from internal nodes
def negative_sampling_internal(pos_edge_index, n_src, n_tar, used, int_drug_index, device=None):
    pos = pos_edge_index.clone().cpu().numpy()
    pos_pair = set([(l[0], l[1]) for l in pos.T.tolist()])
    
    int_drugs = set(int_drug_index)
    neg_tars = [0]*pos.shape[1]
    neg_srcs = [0]*pos.shape[1]
    
    # Choice random disease
    for i, src in enumerate(pos[0]):
        while True:
            neg_tar = np.random.randint(0, n_tar)
            if used is not None:
                if {(src, neg_tar)} <= used:
                    continue
            if not {(src, neg_tar)} <= pos_pair:
                break
        neg_tars[i] = neg_tar
    
    # Choice random drugs
    for i, tar in enumerate(pos[1]):
        while True:
            neg_src = np.random.randint(0, n_src)
            if neg_src not in int_drugs:
                continue
            if used is not None:
                if {(neg_src, tar)} <= used:
                    continue
            if not {(neg_src, tar)} <= pos_pair:
                break
        neg_srcs[i] = neg_src
    
    neg_edge = Tensor(np.concatenate([np.array([pos[0], neg_tars]), np.array([neg_srcs, pos[1]])], axis=1)).long()
    if device:
        neg_edge = neg_edge.to(device)
    
    return neg_edge

In [24]:
def negative_sampling_external(pos_edge_index, n_tar, used, device=None, negative_size=2):
    
    # choice negative pair from only disease nodes
    pos = pos_edge_index.clone().cpu().numpy()
    pos_pair = set([(l[0], l[1]) for l in pos.T.tolist()])
    neg_pairs = []
    
    # Choice random disease
    for i, src in enumerate(pos[0]):
        for _ in range(negative_size):
            while True:
                neg_tar = np.random.randint(0, n_tar)
                if {(src, neg_tar)} <= used:
                    continue
                if not {(src, neg_tar)} <= pos_pair:
                    break
            neg_pairs.append([src, neg_tar])
            
    neg_edge = Tensor(np.array(neg_pairs)).T.long()
    if device:
        neg_edge = neg_edge.to(device)
    
    return neg_edge

In [25]:
def AUPR(prob, y):
    precision, recall, thresh = precision_recall_curve(y, prob)
    auprc = auc(recall, precision)
    return auprc

In [26]:
def AUROC(prob, y):
    fpr, tpr, thresh = roc_curve(y, prob)
    auroc = auc(fpr, tpr)
    return auroc

In [27]:
def ACC(prob, y):
    pred = (prob.flatten() >= 0.5).astype(int)
    acc = accuracy_score(pred, y)
    return acc

In [28]:
def calc_metrics(prob, y):
    auprc = AUPR(prob, y)
    auroc = AUROC(prob, y)
    acc = ACC(prob, y)
    ret = {
        'AUROC' : auroc,
        'AUPRC' : auprc,
        'ACC' : acc
    }
    return ret

In [29]:
def evaluation(pred, target):
    metrics = calc_metrics(pred, target)
    return metrics

In [30]:
def calc_cos_sim(v1,v2):
    return np.dot(v1,v2) / (np.linalg.norm(v1)*np.linalg.norm(v2))

In [31]:
def show_max_values(auroc, auprc, acc):
    print("AUROC: {}".format(max(auroc)))
    print("AUPRC: {}".format(max(auprc)))
    print("ACC: {}".format(max(acc)))

In [32]:
def calc_average_and_std(value):
    ret = np.array([max(v) for v in value])
    ave = np.mean(ret)
    std = np.std(ret)
    return ave, std

In [35]:
# nodes
node_names = ['drug', 'disease']
node_symbols = {
    'drug': 'd',
    'disease': 'z'
}
symbol_to_nodename = {v:k for k,v in node_symbols.items()}
nodes = load_nodes('..', node_names)

In [36]:
# unweighted edges
unweighted_edge_names = [
    'drug_disease',
    'drug_drug'
]
weighted_edge_names = [
    'disease_disease'
]
edge_symbols = {
    'disease_disease': 'z_z',
    'drug_disease': 'd_z',
    'drug_drug': 'd_d'
}
symbol_to_edgename = {v:k for k,v in edge_symbols.items()}
edges, edge_weights = load_edges('..', unweighted_edge_names, weighted_edge_names)

In [37]:
conf = {
    # configuration: model definition parameters
    'target_network': 'drug_disease',
    'task_type': 'transductive', # 'transductive' or 'inductive'
    'emb_loss': 'none', # 'cos' or 'none'
    'enc_type': 'MIX', # "MIX" or "GNN" or "NN" or "RES"
    'dec_type': 'CAT', # "CAT" or "MUL"

    # configuration: training confition parameters
    'reg_type': 1,
    'lr': 0.01,
    'emb_dim': 128,
    'cycle_num': 2,
    'norm_lambda': 1e-7,
    'dropedge_ratio': 0.2,
    'dropout_ratio': 0.5,

    # configuration: other parameters
    'cuda': 7,
    'cv': 5,
    'eps': 1e-6,
    'epoch_num': 5000,
    'verbose': 10,
}

In [38]:
data = Data.from_dict(dict())

In [39]:
# load node counts
for node in node_names:
    data['n_{}'.format(node)] = count_unique_ids(nodes[node])

In [40]:
# load edge indexes
for network in edge_symbols.keys():
    data['{}_edge_index'.format(edge_symbols[network])] = edges[network]

In [41]:
# data split
data = split_link_prediction_datas(data, nodes, edge_symbols, conf['task_type'], conf['cv'])

Transductive link prediction


In [42]:
# "Positive samples" add to used pair
raw_used = get_pair(data.d_z_edge_index)

In [43]:
if conf['task_type'] == 'inductive':
    
    # Negative samplingg for external edges
    data.valid_neg_edge_index = negative_sampling_external(
        data.valid_edge_index, data.n_disease, raw_used, conf['device'], negative_size=2
    )

    data.test_neg_edge_index = negative_sampling_external(
        data.test_edge_index, data.n_disease, raw_used, conf['device'], negative_size=2
    )
    """
    data.valid_neg_edge_index = negative_sampling_external_v2(
        data.valid_edge_index, data.n_disease, used, data.ext_drug_index, conf['device']
    )

    data.test_neg_edge_index = negative_sampling_external_v2(
        data.test_edge_index, data.n_disease, used, data.ext_drug_index, conf['device']
    )
    """
    
elif conf['task_type'] == 'transductive':
    data.valid_neg_edge_index = []
    for i in range(conf['cv']):
        valid_neg_edge_index = negative_sampling_internal(
            data.valid_edge_index[i], data.n_drug, data.n_disease, raw_used, data.int_drug_index
        )
        data.valid_neg_edge_index.append(valid_neg_edge_index)
    
    data.test_neg_edge_index = negative_sampling_internal(
        data.test_edge_index, data.n_drug, data.n_disease, raw_used, data.int_drug_index
    )
    
else:
    raise Exception("Task type error")

In [44]:
# "Negative samples for valid/test data" add to used pair
used = []
for i in range(conf['cv']):
    cv_used = copy.deepcopy(raw_used)
    neg_pair = get_pair(data.valid_neg_edge_index[i])
    cv_used = union_set(cv_used, neg_pair)

    neg_pair = get_pair(data.test_neg_edge_index)
    cv_used = union_set(cv_used, neg_pair)
    used.append(cv_used)

# Main.py

In [45]:
@torch.no_grad()
def train_test_fn(model, train_loader, val_loader, save_file_format=None):
    device = model.device
    state = model.training
    model.eval()
    scores, labels, edges = [], [], []
    for batch in train_loader:
        model.train_step(batch)
    for batch in val_loader:
        batch = move_data_to_device(batch, device)
        output = model.test_step(batch)
        label, score = output["label"], output["predict"]
        edge = batch.interaction_pair[:, batch.valid_mask.reshape(-1)]
        scores.append(score.detach().cpu())
        labels.append(label.cpu())
        edges.append(edge.cpu())
    model.train(state)
    scores = torch.cat(scores).numpy()
    labels = torch.cat(labels).numpy()
    edges = torch.cat(edges, dim=1).numpy()
    eval_star_time_stamp = time.time()
    metric = metric_fn.evaluate(predict=scores, label=labels)
    eval_end_time_stamp = time.time()
    logger.info(f"eval time cost: {eval_end_time_stamp - eval_star_time_stamp}")
    if save_file_format is not None:
        save_file = save_file_format.format(aupr=metric["aupr"], auroc=metric["auroc"])
        scio.savemat(save_file, {"row": edges[0],
                                 "col": edges[1],
                                 "score": scores,
                                 "label": labels,
                                 })
        logger.info(f"save time cost: {time.time() - eval_end_time_stamp}")
    return scores, labels, edges, metric

In [46]:
@torch.no_grad()
def test_fn(model, val_loader, save_file_format=None):
    device = model.device
    state = model.training
    model.eval()
    scores, labels, edges = [], [], []
    for batch in val_loader:
        batch = move_data_to_device(batch, device)
        output = model.step(batch)
        label, score = output["label"], output["predict"]
        edge = batch.interaction_pair[:, batch.valid_mask.reshape(-1)]
        scores.append(score.detach().cpu())
        labels.append(label.cpu())
        edges.append(edge.cpu())
    model.train(state)
    scores = torch.cat(scores).numpy()
    labels = torch.cat(labels).numpy()
    edges = torch.cat(edges, dim=1).numpy()
    eval_star_time_stamp = time.time()
    metric = metric_fn.evaluate(predict=scores, label=labels)
    eval_end_time_stamp = time.time()
    logger.info(f"eval time cost: {eval_end_time_stamp-eval_star_time_stamp}")
    if save_file_format is not None:
        save_file = save_file_format.format(aupr=metric["aupr"], auroc=metric["auroc"])
        scio.savemat(save_file, {"row": edges[0],
                      "col": edges[1],
                      "score": scores,
                      "label": labels,
                      })
        logger.info(f"save time cost: {time.time()-eval_end_time_stamp}")
    return scores, labels, edges, metric

In [47]:
def train_fn(config, model, train_loader, val_loader):
    checkpoint_callback = ModelCheckpoint(monitor="val/auroc",
                                          mode="max",
                                          save_top_k=1,
                                          verbose=False,
                                          save_last=True)
    lr_callback = pl.callbacks.LearningRateMonitor("epoch")
    trainer = Trainer(max_epochs=config.epochs,
                      default_root_dir=config.log_dir,
                      profiler=config.profiler,
                      fast_dev_run=False,
                      enable_checkpointing=checkpoint_callback,
                      callbacks=[lr_callback],
                      gpus=config.gpus,
                      check_val_every_n_epoch=1
                      )
    trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
    if not hasattr(config, "dirpath"):
        config.dirpath = trainer.checkpoint_callback.dirpath
    # checkpoint and add path
    # checkpoint = torch.load("lightning_logs/version_7/checkpoints/epoch=85.ckpt")
    # trainer.on_load_checkpoint(checkpoint)
    print(model.device)

In [48]:
def train(config, model_cls=DRHGCN):
    time_stamp = time.asctime()
    datasets = DRDataset(
        dataset_name=config.dataset_name,
        drug_neighbor_num=config.drug_neighbor_num,
        disease_neighbor_num=config.disease_neighbor_num
    )
    log_dir = os.path.join(f"{config.comment}", f"{config.split_mode}-{config.n_splits}-fold", f"{config.dataset_name}",
                           f"{config.seed}",f"{model_cls.__name__}", f"{time_stamp}")
    config.log_dir = log_dir
    config.n_drug = datasets.drug_num
    config.n_disease = datasets.disease_num

    config.size_u = datasets.drug_num
    config.size_v = datasets.disease_num

    config.gpus = 1 if torch.cuda.is_available() else 0
    config.pos_weight = datasets.pos_weight
    config.time_stamp = time_stamp
    logger = init_logger(log_dir)
    logger.info(pformat(vars(config)))
    config.dataset_type = config.dataset_dype if config.dataset_type is not None else model_cls.DATASET_TYPE
    cv_spliter = CVDataset(datasets, split_mode=config.split_mode, n_splits=config.n_splits,
                           drug_idx=config.drug_idx, disease_idx=config.disease_idx,
                           train_fill_unknown=config.train_fill_unknown,
                           global_test_all_zero=config.global_test_all_zero, seed=config.seed,
                           dataset_type=config.dataset_type)
    pl.seed_everything(config.seed)
    scores, labels, edges, split_idxs = [], [], [], []
    metrics = {}
    start_time_stamp = time.time()
    for split_id, datamodule in enumerate(cv_spliter):
        # if split_id not in [4, 5]:
        #     continue
        config.split_id = split_id
        split_start_time_stamp = time.time()

        datamodule.prepare_data()
        train_loader = datamodule.train_dataloader()
        val_loader = datamodule.val_dataloader()
        config.pos_weight = train_loader.dataset.pos_weight
        model = model_cls(**vars(config))
        model = model.cuda() if config.gpus else model

        if split_id==0:
            logger.info(model)
        logger.info(f"begin train fold {split_id}/{len(cv_spliter)}")
        train_fn(config, model, train_loader=train_loader, val_loader=val_loader)
        logger.info(f"end train fold {split_id}/{len(cv_spliter)}")
        save_file_format = os.path.join(config.log_dir,
                                        f"{config.dataset_name}-{config.split_id} fold-{{auroc}}-{{aupr}}.mat")
        score, label, edge, metric = test_fn(model, val_loader, save_file_format)
        # score, label, edge, metric = train_test_fn(model, train_loader, val_loader, save_file_format)
        metrics[f"{split_id}"] = metric
        scores.append(score)
        labels.append(label)
        edges.append(edge)
        split_idxs.append(np.ones(len(score), dtype=int)*split_id)
        logger.info(f"{split_id}/{len(cv_spliter)} folds: {metric}")
        logger.info(f"{split_id}/{len(cv_spliter)} folds time cost: {time.time()-split_start_time_stamp}")

        if config.debug:
            break
    end_time_stamp = time.time()
    logger.info(f"total time cost:{end_time_stamp-start_time_stamp}")
    scores = np.concatenate(scores, axis=0)
    labels = np.concatenate(labels, axis=0)
    edges = np.concatenate(edges, axis=1)
    split_idxs = np.concatenate(split_idxs, axis=0)
    final_metric = metric_fn.evaluate(predict=scores, label=labels, is_final=True)
    metrics["final"] = final_metric
    metrics = pd.DataFrame(metrics).T
    metrics.index.name = "split_id"
    metrics["seed"] = config.seed
    logger.info(f"final {config.dataset_name}-{config.split_mode}-{config.n_splits}-fold-auroc:{final_metric['auroc']}-aupr:{final_metric['aupr']}")
    output_file_name = f"final-{config.dataset_name}-{config.split_mode}-{config.n_splits}-auroc:{final_metric['auroc']}-aupr:{final_metric['aupr']}-fold"
    scio.savemat(os.path.join(log_dir, f"{output_file_name}.mat"),
                 {"row": edges[0],
                  "col": edges[1],
                  "score": scores,
                  "label": labels,
                  "split_idx":split_idxs}
                 )
    with pd.ExcelWriter(os.path.join(log_dir, f"{output_file_name}.xlsx")) as f:
        metrics.to_excel(f, sheet_name="metrics")
        params = pd.DataFrame({key:str(value) for key, value in vars(config).items()}, index=[str(time.time())])
        for key, value in final_metric.items():
            params[key] = value
        params["file"] = output_file_name
        params.to_excel(f, sheet_name="params")

    logger.info(f"save final results to r'{os.path.join(log_dir, output_file_name)}.mat'")
    logger.info(f"final results: {final_metric}")

In [49]:
def pathIterator(comment):
    split_modes = sorted(os.listdir(comment))
    for split_mode in split_modes:
        for dataset in os.listdir(os.path.join(comment, split_mode)):
            for model in os.listdir(os.path.join(comment, split_mode, dataset)):
                for seed in os.listdir(os.path.join(comment, split_mode, dataset, model)):
                    for time_stamp in os.listdir(os.path.join(comment, split_mode, dataset, model, seed)):
                        yield comment, split_mode, dataset, model, seed, time_stamp

In [50]:
def report(comment, collect_parma=True):
    metrics = []
    params = []
    for comment, split_mode, dataset, model, seed, time_stamp in pathIterator(comment):
        dataset_dir = os.path.join(comment, split_mode, dataset, model, seed, time_stamp)
        valid_files = sorted([ file for file in os.listdir(dataset_dir) if file.endswith(".xlsx") and file.startswith("final")])
        for file in tqdm(valid_files):
            with pd.ExcelFile(os.path.join(dataset_dir, file)) as reader:
                metric = reader.parse(reader.sheet_names[0], index_col=0)
                if collect_parma:
                    param = reader.parse("params", index_col=0)
                    params.append(param)
                    metric["index"] = param.index[0]
                    metric["seed"] = param["seed"].iloc[0]
                metric["model"] = model
                metric["split_mode"] = split_mode
                metric["dataset"] = dataset
                metric["comment"] = comment
                metrics.append(metric)
    metrics = pd.concat(metrics)
    metrics.index.name = "split_id"
    if collect_parma:
        params = pd.concat(params)
    with pd.ExcelWriter(f"{comment}.xlsx") as writer:
        metrics.to_excel(writer, sheet_name="metrics")
        if collect_parma:
            params.to_excel(writer, sheet_name="params")

In [51]:
def parse(conf, print_help=False):
    parser = argparse.ArgumentParser()
    parser.add_argument("--model", default="DRHGCN", type=str)
    parser.add_argument("--epochs", default=50, type=int)
    parser.add_argument("--debug", default=False, action="store_true")
    parser.add_argument("--profiler", default=False, type=str)
    parser.add_argument("--comment", default="runs", type=str, help="experiment name")
    parser = DRDataset.add_argparse_args(parser)
    parser = CVDataset.add_argparse_args(parser)
    parser = DRHGCN.add_model_specific_args(parser)
    args = parser.parse_args(args=conf)
    if print_help:
        parser.print_help()
    return args

In [52]:
conf = ['--epochs', '1000', '--n_splits', '5', '--dataset_name', 'mydataset']
args = parse(conf, print_help=False)
config = parse(conf, print_help=False)
model_cls = DRHGCN
vars(args)

{'model': 'DRHGCN',
 'epochs': 1000,
 'debug': False,
 'profiler': False,
 'comment': 'runs',
 'dataset_name': 'mydataset',
 'drug_neighbor_num': 15,
 'disease_neighbor_num': 15,
 'split_mode': 'global',
 'n_splits': 5,
 'drug_idx': None,
 'disease_idx': None,
 'global_test_all_zero': False,
 'train_fill_unknown': True,
 'dataset_type': None,
 'seed': 666,
 'embedding_dim': 128,
 'layer_num': 2,
 'lr': 0.01,
 'dropout': 0.4,
 'edge_dropout': 0.2,
 'neighbor_num': 15,
 'smooth': False,
 'gnn_mode': 'gcnt'}

In [53]:
def remove_test_edge_from_interaction(data, datasets):
    for [i,j] in data.test_edge_index.numpy().T:
        datasets.interactions[j][i] = 0

In [54]:
@torch.no_grad()
def test_fn_formy(model, val_loader, save_file_format=None):
    device = model.device
    state = model.training
    model.eval()
    scores, labels, edges = [], [], []
    for batch in val_loader:
        batch = move_data_to_device(batch, device)
        output = model.step_formy(batch)
        label, score = output["label"], output["predict"]
        edge = batch.interaction_pair[:, batch.valid_mask.reshape(-1)]
        scores.append(score.detach().cpu())
        labels.append(label.cpu())
        edges.append(edge.cpu())
    model.train(state)
    scores = torch.cat(scores).numpy()
    labels = torch.cat(labels).numpy()
    edges = torch.cat(edges, dim=1).numpy()
    eval_star_time_stamp = time.time()
    metric = metric_fn.evaluate(predict=scores, label=labels)
    eval_end_time_stamp = time.time()
    if save_file_format is not None:
        save_file = save_file_format.format(aupr=metric["aupr"], auroc=metric["auroc"])
        scio.savemat(save_file, {"row": edges[0],
                      "col": edges[1],
                      "score": scores,
                      "label": labels,
                      })
        logger.info(f"save time cost: {time.time()-eval_end_time_stamp}")
    return scores, labels, edges, metric, output

In [55]:
def eval_mytest(result, data):
    score = result['predict_raw'].reshape([data.n_disease, data.n_drug])
    pred, label = [], []
    # positive
    for [i,j] in data.test_edge_index.numpy().T:
        pred.append(score[j][i])
        label.append(1)
    
    # negative
    for [i,j] in data.test_neg_edge_index.numpy().T:
        pred.append(score[j][i])
        label.append(0)
    
    pred = np.array(pred)
    label = np.array(label)
    metrics = evaluation(pred, label)
    return metrics

In [56]:
def train_formy(config, model_cls=DRHGCN):
    time_stamp = time.asctime()
    test_scores = []
    predicted_mats = []
    datasets = DRDataset(
        dataset_name=config.dataset_name,
        drug_neighbor_num=config.drug_neighbor_num,
        disease_neighbor_num=config.disease_neighbor_num
    )
    remove_test_edge_from_interaction(data, datasets)
    log_dir = os.path.join(f"{config.comment}", f"{config.split_mode}-{config.n_splits}-fold", f"{config.dataset_name}",
                           f"{config.seed}",f"{model_cls.__name__}", f"{time_stamp}")
    config.log_dir = log_dir
    config.n_drug = datasets.drug_num
    config.n_disease = datasets.disease_num

    config.size_u = datasets.drug_num
    config.size_v = datasets.disease_num

    config.gpus = 1 if torch.cuda.is_available() else 0
    config.pos_weight = datasets.pos_weight
    config.time_stamp = time_stamp
    config.dataset_type = config.dataset_dype if config.dataset_type is not None else model_cls.DATASET_TYPE
    cv_spliter = CVDataset(datasets, split_mode=config.split_mode, n_splits=config.n_splits,
                           drug_idx=config.drug_idx, disease_idx=config.disease_idx,
                           train_fill_unknown=config.train_fill_unknown,
                           global_test_all_zero=config.global_test_all_zero, seed=config.seed,
                           dataset_type=config.dataset_type)
    pl.seed_everything(config.seed)
    scores, labels, edges, split_idxs = [], [], [], []
    metrics = {}
    start_time_stamp = time.time()
    for split_id, datamodule in enumerate(cv_spliter):
        # if split_id not in [4, 5]:
        #     continue
        config.split_id = split_id
        split_start_time_stamp = time.time()

        datamodule.prepare_data()
        train_loader = datamodule.train_dataloader()
        val_loader = datamodule.val_dataloader()
        config.pos_weight = train_loader.dataset.pos_weight
        model = model_cls(**vars(config))
        model = model.cuda() if config.gpus else model

        if split_id==0:
            logger.info(model)
        train_fn(config, model, train_loader=train_loader, val_loader=val_loader)
        score, label, edge, metric, result = test_fn_formy(model, val_loader)
        # score, label, edge, metric = train_test_fn(model, train_loader, val_loader, save_file_format)
        metrics[f"{split_id}"] = metric
        scores.append(score)
        labels.append(label)
        edges.append(edge)
        split_idxs.append(np.ones(len(score), dtype=int)*split_id)

        if config.debug:
            break
            
        test_score = eval_mytest(result, data)
        test_scores.append(test_score)
        predicted_mats.append(result['predict_raw'].reshape([data.n_disease, data.n_drug]))
            
    end_time_stamp = time.time()
    scores = np.concatenate(scores, axis=0)
    labels = np.concatenate(labels, axis=0)
    edges = np.concatenate(edges, axis=1)
    split_idxs = np.concatenate(split_idxs, axis=0)
    final_metric = metric_fn.evaluate(predict=scores, label=labels, is_final=True)
    metrics["final"] = final_metric
    metrics = pd.DataFrame(metrics).T
    metrics.index.name = "split_id"
    metrics["seed"] = config.seed
    
    return metrics, test_scores, result

In [57]:
val_metrics, test_metrics, predicted_mats = train_formy(args)

Global seed set to 666


dataset:mydataset, drug:4446, disease:1870, pos weight:698.5977785257489


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: runs/global-5-fold/mydataset/666/DRHGCN/Wed Sep 28 02:19:48 2022/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name            | Type                | Params
--------------------------------------------------------
0 | edge_dropout    | EdgeDropout         | 0     
1 | short_embedding | Linear              | 808 K 
2 | intra_encoders  | ModuleList          | 1.7 M 
3 | inter_encoders  | ModuleList          | 825 K 
4 | decoder         | InnerProductDecoder | 0     
5 | smooth_decoder  | SmoothDecoder       | 0     
--------------------------------------------------------
3.3 M     Trainable params
0         Non-trainable params
3.3 M     Total params
13.135    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1000` reached.


cpu


In [58]:
val_metrics

,aupr,auroc,lagcn_aupr,lagcn_auc,lagcn_f1_score,lagcn_accuracy,lagcn_recall,lagcn_specificity,lagcn_precision,seed
split_id,,,,,,,,,,
0,0.038637,0.935092,0.089237,0.935069,0.105246,0.996707,0.177313,0.997603,0.074832,666
1,0.032057,0.934472,0.086115,0.934433,0.089002,0.996110,0.174009,0.997008,0.059792,666
2,0.036733,0.933913,0.108624,0.933868,0.102055,0.996846,0.164097,0.997757,0.074056,666
3,0.036070,0.939318,0.081717,0.939324,0.095578,0.996347,0.176859,0.997242,0.065483,666
4,0.035105,0.932785,0.098899,0.932755,0.095584,0.997007,0.144904,0.997938,0.071312,666
final,0.035519,0.935119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,666


In [59]:
best_idx = np.argmax(val_metrics['aupr'].values)
print("Best index: {}".format(best_idx))h

Best index: 0


In [60]:
test_metrics[best_idx]

{'AUROC': 0.7627620249942415,
 'AUPRC': 0.5611784152998253,
 'ACC': 0.6591830371063299}